In [1]:
import os
os.environ["WANDB_DISABLED"] = "true"

import sys
sys.path.append('..')

%load_ext autoreload
%autoreload 2

In [2]:
import analyze_utils

checkpoint_folder = "/home/jxm3/research/retrieval/inversion/saves/ef674b4d1bdb4d5d5d678e8ef1aafe98/"
trainer = analyze_utils.load_trainer(checkpoint_folder)

Loading trainer for analysis – setting --do_eval=1
loading alias dpr_nq__msl32_beta from /home/jxm3/research/retrieval/inversion/saves/db66b9c01b644541fedbdcc59c53a285/ebb31d91810c4b62d2b55b5382e8c7ea...
Set train_args.dataloader_num_workers = 4


/home/jxm3/.conda/envs/torch/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


================ Begin trainer sanity check ================
	Input to encode -> Twas brillig, and the slithy toves, Did gyre and gimble in the wabe, All mimsy were the borogoves, And the mome raths outgrabe.
	Decoded output -> <pad> The m<unk>lbies wase wyst bograge; And the sliths and toms wy</s>
================ End trainer sanity check ================


In [30]:
trainer.args.per_device_eval_batch_size = 64
eval_batch = next(iter(trainer.get_eval_dataloader()))
# eval_batch = {k: v[0:1, ...] for k,v in eval_batch.items()} # just one guy
eval_batch = {k: v.to(trainer.args.device) for k,v in eval_batch.items()}

In [31]:
trainer.embedder_tokenizer.batch_decode(eval_batch['embedder_input_ids'])

['to the character of the sutures of the skull which, like those of the infant skull, are relatively smooth and untortuous. In fact</s>',
 "individual from the Southern Hemisphere, to win a Winter Olympic gold medal and was also part of the short track relay team that won Australia'</s>",
 'the same rights as straight people, while 15% disagreed. Additionally, 69% agreed that they should be protected from workplace discrimination. 13% of H</s>',
 'Bruthen, Victoria Bruthen is a small town located alongside the Tambo River between Bairnsdale and Ensay on</s>',
 'Castle Vale Castle Vale is a housing estate located between Erdington, Minworth and Castle Bromwich. Currently Castle Vale votes</s>',
 '1839. After her death, Fisk married her sister, Simona, on August 1, 1849. Deaf Smith moved freely between Anglo</s>',
 "in 1978 he was a member of the Soviet Composer's Society in Moscow. He obtained commissions from the Russian Broadcasting Corporation</s>",
 'Zachary Throne Zachary Throne (b

In [83]:
import torch
with torch.no_grad():
    reranked_gens = trainer.generate_with_reranking(eval_batch, L=5, B=4)

[2, 2, 1, 2, 1, 1, 3, 0, 0, 3, 1, 1, 1, 1, 3, 2, 1, 2, 1, 0, 1, 3, 3, 2, 0, 1, 2, 2, 1, 0, 0, 1, 2, 3, 3, 1, 0, 1, 1, 1, 2, 0, 3, 2, 2, 0, 3, 1, 1, 0, 1, 1, 2, 2, 0, 1, 0, 3, 1, 0, 2, 0, 3, 0]
[2, 3, 2, 2, 2, 1, 3, 1, 3, 0, 2, 2, 2, 2, 1, 2, 2, 3, 3, 1, 2, 2, 1, 1, 3, 0, 3, 1, 0, 1, 1, 2, 2, 1, 1, 3, 3, 2, 3, 3, 0, 0, 2, 1, 3, 3, 0, 3, 0, 0, 2, 0, 3, 3, 3, 0, 2, 1, 3, 0, 2, 3, 0, 1]
[2, 0, 0, 2, 3, 3, 2, 2, 1, 2, 3, 2, 0, 3, 0, 3, 1, 0, 0, 1, 1, 1, 0, 3, 0, 1, 2, 1, 3, 2, 2, 3, 2, 1, 3, 2, 1, 0, 0, 1, 0, 3, 0, 2, 0, 3, 3, 2, 3, 1, 0, 2, 0, 1, 3, 2, 3, 0, 3, 1, 3, 3, 1, 3]
[2, 2, 3, 2, 3, 0, 0, 2, 1, 2, 0, 0, 1, 2, 1, 3, 1, 1, 2, 2, 3, 1, 1, 3, 1, 2, 3, 3, 1, 2, 3, 1, 1, 3, 3, 1, 0, 0, 0, 0, 2, 1, 0, 0, 3, 2, 2, 1, 2, 3, 3, 3, 0, 2, 0, 3, 2, 0, 3, 1, 2, 1, 1, 1]
[3, 0, 0, 1, 0, 2, 0, 2, 3, 3, 2, 3, 1, 1, 3, 0, 1, 2, 3, 2, 2, 1, 1, 1, 3, 3, 2, 3, 0, 3, 0, 3, 2, 1, 2, 3, 0, 1, 0, 1, 0, 3, 0, 0, 2, 0, 0, 2, 3, 3, 0, 3, 2, 2, 1, 2, 0, 2, 3, 2, 1, 1, 1, 2]
[0, 3, 3, 3, 2, 0, 2, 3, 3, 0, 0, 3

In [84]:
gens = trainer.inversion_trainer.model.generate(eval_batch, generation_kwargs={
    "num_beams": 4,
    "num_return_sequences": 1,
    "do_sample": False,
    "no_repeat_ngram_size": 3,
})[:, 1:]

In [85]:
reranked_gens.shape, gens.shape

(torch.Size([64, 32]), torch.Size([64, 32]))

In [86]:
trainer.embedder_tokenizer.batch_decode(gens[0:1])

['to that of the skulls of infant sutures, which are relatively smooth and untidy in character. Nevertheless, the suture </s>']

In [87]:
trainer.embedder_tokenizer.batch_decode(reranked_gens[0:1])

['to that of the skull, which are relatively smooth in character and untutuous in the case of sutures of infant skulls. Nevertheless</s>']

In [88]:
gens_text = trainer.embedder_tokenizer.batch_decode(gens, add_special_tokens=False)
reranked_text = trainer.embedder_tokenizer.batch_decode(reranked_gens, add_special_tokens=False)
true_text = trainer.embedder_tokenizer.batch_decode(eval_batch['embedder_input_ids'])

In [89]:
bleu = trainer.inversion_trainer.metric_bleu

In [90]:
bleu.compute(
    predictions=gens_text, references=true_text
)

{'score': 35.64198521987842,
 'counts': [1296, 765, 481, 305],
 'totals': [1814, 1750, 1686, 1622],
 'precisions': [71.44432194046307,
  43.714285714285715,
  28.52906287069988,
  18.803945745992603],
 'bp': 0.990672221585101,
 'sys_len': 1814,
 'ref_len': 1831}

In [91]:
bleu.compute(
    predictions=reranked_text, references=true_text
)

{'score': 28.030885101832112,
 'counts': [1245, 657, 371, 188],
 'totals': [1841, 1777, 1713, 1649],
 'precisions': [67.62629005975013,
  36.972425436128304,
  21.657910099241096,
  11.400848999393572],
 'bp': 1.0,
 'sys_len': 1841,
 'ref_len': 1831}

## looking at generations

In [13]:
true_ids = eval_batch['embedder_input_ids']
(true_ids[:, 0] == gens[:, 0]).float().mean()

tensor(1., device='cuda:0')

In [14]:
(true_ids[:, 0] == reranked_gens[:, 0]).float().mean()

tensor(1., device='cuda:0')

In [15]:
import pandas as pd

df = pd.DataFrame(zip(true_text, gens_text, reranked_text), columns=['true', 'generated', 'generated w/ reranking'])

In [16]:
df

,true,generated,generated w/ reranking
0,to the character of the sutures of the skull w...,"to the character of the skull, which are relat...","to that of the skulls of infant sutures, which..."
